In [1]:
import subprocess
import shlex
import os
import glob
import re

from subprocess import PIPE
from pathlib import Path

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [3]:
def submit_independent_spatially_explicit(
    delta_t, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent(
            delta_t: {delta_t},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [4]:
def submit_cuda_spatially_explicit(
    delta_t, block_size, grid_size, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={24}gb:ngpus={1}:gpu_type=P1000
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-cuda/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: CUDA(
            device: {0},
            ptx_jit: {str(True).lower()},
            delta_t: {delta_t},
            block_size: {block_size},
            grid_size: {grid_size},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [5]:
def submit_monolithic_spatially_explicit(
    algorithm,
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if content == '':
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: {algorithm}(
            parallelism_mode: Monolithic,
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [6]:
def submit_necsim(
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    glob_pathname = f"{glob.escape(output)}.e*.*"
    
    successful = []
    erroneous = []
    
    for path in glob.iglob(glob_pathname):
        with open(path) as file:
            content = file.read()
            
            if "Simulation time was" in content:
                successful.append(path)
            else:
                erroneous.append(path)
                
    for error_path in erroneous:
        Path('o'.join(error_path.rsplit('e', 1))).unlink()
        Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    repeats = max(repeats, 2)
    
    walltime = "{:02d}".format(walltime)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    module load anaconda3/personal

    python3 -c '
import logging
import random

from pycoalescence import Simulation, Map

source_dir = "'$HOME'/necsim-rust/maps/madingley/fg0size12/"

s = Simulation(logging_level=logging.INFO)

s.set_simulation_parameters(
    seed=random.getrandbits(30), task=0, output_directory="'$TMPDIR'", min_speciation_rate={speciation},
    deme=1, sample_size={sample}, max_time=60*60*72
)
s.set_map_files(sample_file="null", fine_file=(source_dir+"habitat.tif"), dispersal_map=(source_dir+"dispersal.tif"))

s.run()
    '
    """
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [7]:
for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_monolithic_spatially_explicit(
        "Classical",
        walltime=12, repeats=10, speciation=speciation, sample=0.00025,
        output=f"classical/pbs.{speciation}",
    )

for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_monolithic_spatially_explicit(
        "Gillespie",
        walltime=16, repeats=10, speciation=speciation, sample=0.00025,
        output=f"gillespie/pbs.{speciation}",
    )
        
for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_monolithic_spatially_explicit(
        "SkippingGillespie",
        walltime=16, repeats=10, speciation=speciation, sample=0.00025,
        output=f"skipping-gillespie/pbs.{speciation}",
    )

In [8]:
for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_independent_spatially_explicit(
        2.0, 10, f"Relative(factor: {1.0})", 1000000, # delta_t, step_slice, dedup_cache, event_slice
        walltime=12, repeats=10, speciation=speciation, sample=0.00025,
        output=f"independent/pbs.{speciation}",
    )

In [9]:
for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_cuda_spatially_explicit(
        3.0, 64, 64, 150, f"Relative(factor: {0.1})", 20000000, # delta_t, block_size, grid_size, step_slice, dedup_capacity, event_slice
        walltime=16, repeats=10, speciation=speciation, sample=0.00025,
        output=f"cuda/pbs.{speciation}",
    )

In [10]:
for speciation in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    submit_necsim(
        walltime=16, repeats=10, speciation=speciation, sample=0.00025,
        output=f"necsim/pbs.{speciation}",
    )